# Module2: Boston Housing with Linear Regression

**ボストン市内の特定の地域の住宅価格を予測する回帰式を作成し、どのような地域だと価格が高くなりやすいのかを考察する。**

データには以下の列が含まれています:
* 'crim': 町ごとの一人当たりの犯罪率。
* 'zn': 25,000平方フィートを超える区画に分類される住宅地の割合＝「広い家の割合」
* 'indus': 町ごとの非小売業の面積の割合。
* 'chas':チャールズ川のダミー変数（区画が川に接している場合は1、そうでない場合は0）＝「川の隣か」
* 'nox': 「NOx濃度（0.1ppm単位）」＝一酸化窒素濃度（parts per 10 million単位）。この項目を目的変数とする場合もある
* 'rm': 住戸あたりの平均部屋数。
* 'age': 1940年より前に建てられた持ち家の割合＝「古い家の割合」
* 'dis': ボストンの 5 つの雇用センターまでの距離の加重平均＝「主要施設への距離」
* 'rad': 放射状の高速道路へのアクセス性の指標。
* 'tax': 1万ドルあたりの全価値固定資産税の税率。
* 'ptratio': 町別の生徒数と教師数の比率。
* 'black':「1000(Bk - 0.63)」の二乗値。Bk＝「町ごとの黒人の割合」を指す
* 'lstat': 「低所得者人口の割合」
* 'medv': 「住宅価格」（1000ドル単位）の中央値。通常はこの数値が目的変数として使われる


## Environment Setup
### 必要なライブラリをインストールする

各セルを1つずつ実行して、必要なライブラリをインストールします。

In [ ]:
!pip install seaborn

互換性に関連するエラーは無視してください。 最後にこのメッセージが表示されている限り正常にインストールされています: Successfully installed seaborn もしくは Requirement already satisfied

## Let's Start !

まず、環境を準備するために、いくつかのライブラリをインポートする必要があります。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import warnings
#Warning を抑止
warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
# データセットをpandasのDataFrameにインポートしてデータを見る
BostonTrain = pd.read_csv("data/boston_all.csv")

**ここでは、Bostonのデータを見てみましょう。**

In [ ]:
#DataFrameの先頭5行分を表示
BostonTrain.head()

In [ ]:
#pandas でデータの要約を表示
BostonTrain.info()

DataFrameの行数、列数、各列の列名、各列に格納されるデータの型、メモリ使用量が表示されます。

In [ ]:
#各列の要約統計量（平均、標準偏差など）を取得
BostonTrain.describe()

**目標はカラムについて考え、どのカラムがモデルを構築するのに関連性があるのかを発見することです。**

## データの前処理

### 欠損値の対応

データをロードした後、データに欠落している値がないかどうかを確認することをお勧めします。isnull() を使用して、各機能の欠落値の数をカウントします。

In [ ]:
BostonTrain.isnull().sum()

In [ ]:
#IDカラムは分析には関係ないので除去します
BostonTrain.drop('ID', axis = 1, inplace=True)

print(BostonTrain.keys())

### 外れ値の対応

データ全体の傾向からかけ離れたデータのことを外れ値と呼びます。外れ値を含んだデータで機械学習を行うと、作成したモデルの予測性能が上がりにくくなってしまうことがあります。ただし、どの外れ値を除外するかは慎重に検討する必要があります。

In [ ]:
BostonTrain.plot.scatter('rm', 'medv', color="tab:blue")

In [ ]:
#rm の外れ値のindexを特定
out_line1 = BostonTrain[(BostonTrain['rm'] > 8) & (BostonTrain['medv'] < 30)].index

out_line2 = BostonTrain[(BostonTrain['rm'] < 5) & (BostonTrain['medv'] > 30)].index

print(out_line1, out_line2)

In [ ]:
#drop メソッドで指定index の行を削除
BostonTrain = BostonTrain.drop(out_line1, axis = 0)
BostonTrain = BostonTrain.drop(out_line2, axis = 0)

In [ ]:
BostonTrain.plot.scatter('rm', 'medv', color="tab:blue")

## 探索的データ分析

探索的データ分析は、モデルをトレーニングする前の非常に重要なステップです。このセクションでは、いくつかの視覚化を使用して、ターゲット変数と他の機能との関係を理解します。

まず、ターゲット変数medv(住宅価格)の分布をプロットしましょう。seabornライブラリの distplot 関数を使用します。

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.distplot(BostonTrain['medv'], bins=30)
plt.show()

medvの値は外れ値がほとんどなく正規分布していることがわかります。

In [ ]:
#住戸あたりの平均部屋数vs「住宅価格」（1000ドル単位）の中央値の散布図
BostonTrain.plot.scatter('rm', 'medv', color="tab:blue")

このプロットでは、明らかに直線的なパターンを見ることができます。住戸あたりの平均部屋数が多いほど、住宅価格の中央値は高くなります。

In [ ]:
sns.pairplot(BostonTrain, diag_kind='kde')

**では、すべての変数がどのように互いに関係しているかを見てみましょう。**

変数間の線形関係を測定する相関行列を作成します。相関行列はcorr()、pandas データフレームライブラリの関数を使用して作成できます。seaborn ライブラリの heatmap 関数を使用して、相関行列をプロットします。

In [ ]:
plt.subplots(figsize=(12,8))
sns.heatmap(BostonTrain.corr(), cmap = 'RdGy', annot=True, fmt="1.2f")

相関係数の範囲は-1から1です。値が1に近い場合は、2つの変数間に強い正の相関があることを意味します。-1に近い場合、変数には強い負の相関があります。

このヒートマッププロットでは、ペアプロットよりも良い分析ができます。
最後の行に注目してみましょう。

- 赤/オレンジの色の濃淡：色が赤いほどX軸上にあり、medvが小さい。負の相関関係
- 淡い色のとき：軸xとyにある変数は、何の関係もありません。相関性ゼロ
- グレー/ブラックの濃淡の場合：X軸上の色が黒いほどmedvの値が高くなります。正の相関関係

In [ ]:
# Variation
plt.figure(figsize=(20,5))
features =['rm', 'lstat']
target = BostonTrain['medv']
for i, col in enumerate(features):
    plt.subplot(1,len(features),i+1)
    x = BostonTrain[col]
    y=target
    plt.scatter(x,y,marker='o')
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel('medv')
    

rm変数は（0.72）であるターゲットmedvと強い相関関係があることがわかりました。 一方、lstatはmedvと高い負の相関があり、（-0.74）です。

線形回帰モデルの特徴を選択する際の重要なポイントは、多重共線性（説明変数の間に強い相関関係が存在する場合）をチェックすることです。特徴量rad、taxの相関は0.91です。これらの特徴量ペアは、互いに強く相関しています。これはモデルに影響を与える可能性があります。-0.75の相関関係を持つ特徴量disとageについても同じことが言えます。これは、多重共線性を除外する必要があり、そのままにするとパラメーターの推定値が不安定になり、目的変数に対する説明変数の影響を評価することが非常に困難になります。


# 線形回帰モデルを訓練する

**XとYの定義**

X：説明変数、独立変数、特徴量として名前が付けられた変数。多群でも可。                       
Y：応答または目的変数として名前が付けられた変数。1群。

In [ ]:
X = BostonTrain[['rm']]
y = BostonTrain['medv']

In [ ]:
#2変数のヒストグラムつきの散布図を作成する
sns.jointplot(x=X['rm'], y='medv', data=BostonTrain, kind='reg')

## データをトレーニングセットとテストセットに分割する

データをトレーニングセットとテストセットに分割します。サンプルの70％でモデルをトレーニングし、残りの30％でテストします。これは、見えないデータに対するモデルのパフォーマンスを評価するために行います。データを分割するには train_test_split、scikit-learnライブラリが提供する関数を使用します。最後に、トレーニングとテストセットのサイズをprintして、分割が適切に行われたかどうかを確認します。

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
# X_train は全データセットの 70% を含む
print(X_train.shape)
# X_test は全データセットの 30% を含む
print(X_test.shape)

## 線形回帰を適用する

scikit-learn ライブラリの LinearRegression を使用して、トレーニングセットとテストセットの両方でモデルをトレーニングします。

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
sns.regplot(y_test,predictions, data=predictions)

## モデル評価

RMSEとR2スコアを使用してモデルを評価します。


In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R-squared:', metrics.r2_score(y_test, predictions))

RMSEを考える：我々は、このモデルの平均誤差がmedvでRMSE=6.1であると結論付けることができます。

- 平均絶対誤差 (MAE, Mean Absolute Error) は、実際の値と予測値の絶対値を平均したものです。MAE が小さいほど誤差が少なく、予測モデルが正確に予測できていることを示し、MAE が大きいほど実際の値と予測値に誤差が大きく、予測モデルが正確に予測できていないといえます。

$$
    MAE=\frac{1}{n}\sum_{i=1}^{n}{|y_i-\hat{y}_i|}   
$$

- 平均二乗誤差 (MSE, Mean Squared Error) とは、実際の値と予測値の絶対値の 2 乗を平均したものです。この為、MAE に比べて大きな誤差が存在するケースで、大きな値を示す特徴があります。MAE と同じく、値が大きいほど誤差の多いモデルと言えます。

$$
    MSE=\frac{1}{n}\sum_{i=1}^{n}{(y_i-\hat{y}_i)^2}
$$


- MSE の平方根を 二乗平均平方根誤差 (RMSE: Root Mean Squared Error) と呼びます。上記の MSE で、二乗したことの影響を平方根で補正したものです。RMSE は、RMSD (Root Mean Square Deviation) と呼ばれることもあります。

$$
    RMSE=\sqrt{\frac{1}{n}\sum_{i=1}^{n}{(y_i-\hat{y}_i)^2}}
$$

- 決定係数 (R2, R-squared, coefficient of determination) は、モデルの当てはまりの良さを示す指標で、最も当てはまりの良い場合、1.0 となります (当てはまりの悪い場合、マイナスとなることもあります)。一般的に決定係数が 0.8 以上であれば、予測性能が高くてよい計算式といわれています。寄与率とも呼ばれます。観測値を $y_i$ ($\mathit{i}$ = 1, 2, 3, ..., $\mathit{n}$)、モデルから計算した計算値（予測値）を $\hat{y}_i$、観測値の平均を $\bar{y}_i$ とすると、

$$
    R^2=1-\frac{\sum_{i=1}^{n}{(y_i-\hat{y}_i)^2}}{\sum_{i=1}^{n}{(y_i-\bar{y}_i)^2}}
$$


対応する二乗平均平方根誤差(RMSE)は約6.1となり、我々のモデルは実際の販売価格を約±$6,100の誤差の範囲で予測できることを表しています。

## 残差の正規性の確認

In [ ]:
sns.distplot((y_test-predictions),bins=50);

残差の正規分布が多ければ多いほど良いモデルといえる。

また、回帰分析の結果から残差を計算しQQ plotを描く。残差のプロットが直線状に乗っている場合は、正規分布していると見ていい。

In [ ]:
import scipy.stats as stats

stats.probplot((y_test-predictions), dist="norm", plot=plt)
plt.show()

残差プロットは、残差（目的変数の真値と予測値の差分）の分布を可視化したものです。線形モデルが目的変数を完璧に予測できる場合は残差は0となるので、予測精度の良い線形モデルの残差プロットは、0を中心にランダムにばらついたものになります。残差プロットに何かパターンが見られる場合は、線形モデルで説明しきれない情報があることが示唆されます。以下のコードは、残差プロットを描画します。

In [ ]:
plt.scatter(predictions, predictions - y_test, color = 'blue')      # 残差をプロット 
plt.hlines(y = 0, xmin = -10, xmax = 50, color = 'black') # x軸に沿った直線をプロット
plt.title('Residual Plot')                                # 図のタイトル
plt.xlabel('Predicted Values')                            # x軸のラベル
plt.ylabel('Residuals')                                   # y軸のラベル
plt.grid()                                                # グリッド線を表示

plt.show()                                               # 図の表示

## すべての特徴量を含めて回帰

In [ ]:
X = BostonTrain[['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'black', 'lstat']]
y = BostonTrain['medv']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
# X_train contains 70% of total dataset
print("X_train size:", X_train.shape, type(X_train), "y_train size:", y_train.shape, type(y_train))
# X_test contains 30% of total dataset
print("X_test size:", X_test.shape, type(X_test), "y_test size:", y_test.shape, type(y_test))

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
sns.regplot(y_test,predictions, data=predictions)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R-squared:', metrics.r2_score(y_test, predictions))

## モデルの保存

In [ ]:
import pickle

with open('BostonPrediction.pkl', 'wb') as f:
    pickle.dump(lm, f)

## モデルのロード

In [ ]:
with open('BostonPrediction.pkl', 'rb') as f:
    model2 = pickle.load(f)

model2

## 残差の正規性の確認

In [ ]:
sns.distplot((y_test-predictions),bins=50)

In [ ]:
import scipy.stats as stats

stats.probplot((y_test-predictions), dist="norm", plot=plt)
plt.show()

In [ ]:
plt.scatter(predictions, predictions - y_test, color = 'blue')      # 残差をプロット 
plt.hlines(y = 0, xmin = -10, xmax = 50, color = 'black') # x軸に沿った直線をプロット
plt.title('Residual Plot')                                # 図のタイトル
plt.xlabel('Predicted Values')                            # x軸のラベル
plt.ylabel('Residuals')                                   # y軸のラベル
plt.grid()                                                # グリッド線を表示

plt.show()                                               # 図の表示

## 特徴量エンジニアリング

元のデータに対して、2乗した値の列や3乗した値の列を生成して特徴量に加えることで特徴量表現が豊かになり、結果的に精度を高められることがあります。このようにして追加した特徴量を多項式特徴量といいます。

In [ ]:
BostonTrain['rm2'] = BostonTrain['rm'] ** 2
BostonTrain['rm3'] = BostonTrain['rm'] ** 3
BostonTrain['rm4'] = BostonTrain['rm'] ** 4
BostonTrain.head()

In [ ]:
X = BostonTrain[['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'black', 'lstat', 'rm2', 'rm3', 'rm4']]
y = BostonTrain['medv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

lm = LinearRegression()
lm.fit(X_train, y_train)

predictions = lm.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R-squared:', metrics.r2_score(y_test, predictions))

また、2つの列の対応する要素同士を掛けることで新たな特徴量を生成することもあります。このようにして追加した特徴量を、交互作用特徴量といいます。

In [ ]:
BostonTrain['rm*lstat'] = BostonTrain['rm'] * BostonTrain['lstat']
BostonTrain.head()

In [ ]:
X = BostonTrain[['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'black', 'lstat', 'rm2',  'rm*lstat']]
y = BostonTrain['medv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

lm = LinearRegression()
lm.fit(X_train, y_train)

predictions = lm.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print('R-squared:', metrics.r2_score(y_test, predictions))